# Basic Ensemble Learning

In [596]:
# Import Required Libraries 
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [597]:
# read the data
df = pd.read_csv('diabetes_data.csv')

In [598]:
df

,pregnancies,glucose,diastolic,triceps,insulin,bmi,dpf,age,diabetes
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [599]:
# check if the columns are in correct data type
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
pregnancies    768 non-null int64
glucose        768 non-null int64
diastolic      768 non-null int64
triceps        768 non-null int64
insulin        768 non-null int64
bmi            768 non-null float64
dpf            768 non-null float64
age            768 non-null int64
diabetes       768 non-null int64
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [600]:
# check shape of data
df.shape

(768, 9)

In [601]:
# Assigning X and y values for the training proccess
X = df.drop('diabetes',axis = 1)
y = df['diabetes']

In [602]:
# importing train_test_split to split the X and y data to x,y(train and test) respectively
from sklearn.model_selection import train_test_split

In [603]:

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.4, 
                                                   random_state=2**31 ,stratify = y)

# Simple Max Voting Ensemble Method 

In [604]:
# importing required models
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

In [605]:
# Assigning models to m1,m2..,m5 respectively
m1 = RandomForestClassifier()
m2 = SVC()
m3 = DecisionTreeClassifier()
m4 = LogisticRegression()
m5 = KNeighborsClassifier()

In [606]:
# Fitting Train data to the models Respectively
m1.fit(X_train,y_train)
m2.fit(X_train,y_train)
m3.fit(X_train,y_train)
m4.fit(X_train,y_train)
m5.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [607]:
# performing predictions off the X_test values
pred1= m1.predict(X_test)
pred2= m2.predict(X_test)
pred3= m3.predict(X_test)
pred4= m4.predict(X_test)
pred5 = m5.predict(X_test)

In [608]:
# creating a data frame for each predictions for better data visualization
df_pred = pd.DataFrame()
df_pred['pred1'] = pred1
df_pred['pred2'] = pred2
df_pred['pred3'] = pred3
df_pred['pred4'] = pred4
df_pred['pred5'] = pred5

In [609]:
# This is a while loop that iterates through the dataframe calculates the mode of the data
"""
since am working with binary numbers(0,1) it is safe to use this method 
but not applicable to non_binary numbers. 
example(1,1,1,0,0) the mode here is 3 and 1+1+1 = 3
unlike(3,3,3,4,5) the mode here is 3 but 3+3+3 = 9
so be aware of the target data.
this code finds the mode and assigns a binary based on the mode
if mode < 3 append(0) to final predictions since the mode here is either 2 or 1
if mode >= 3 append(1) if the mode is high then the probability is high 
"""
i = 0
q= 0
w = df_pred.columns
final_pred = []
while q < len(df_pred):
    for i in df_pred[w[i+1]]:
        if (df_pred[w[i]] + df_pred[w[i+1]] + df_pred[w[i+2]] + df_pred[w[i+3]] +df_pred[w[i+4]])[q] < 3:
            final_pred.append(0)
        elif (df_pred[w[i]] + df_pred[w[i+1]] + df_pred[w[i+2]] + df_pred[w[i+3]] +df_pred[w[i+4]])[q] >= 3:
            final_pred.append(1)
        q = q + 1

In [610]:
len(final_pred)

308

In [611]:
# CLassification_report will be used to score the accuracy of the final_pred based on the original test values
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,final_pred))


              precision    recall  f1-score   support

           0       0.78      0.91      0.84       201
           1       0.74      0.51      0.61       107

    accuracy                           0.77       308
   macro avg       0.76      0.71      0.72       308
weighted avg       0.77      0.77      0.76       308



In [612]:
# checking the individual models for their accuracy
pd.DataFrame(data = [m1.score(X_test,y_test),m2.score(X_test,y_test),m3.score(X_test,y_test),m4.score(X_test,y_test),m5.score(X_test,y_test)]
             ,index=['Model1','Model2','Model3','Model4','Model5'],columns=['Accuracy score'])

,Accuracy score
Model1,0.746753
Model2,0.652597
Model3,0.733766
Model4,0.756494
Model5,0.714286


# The accuracy f1 - score is 77% , the advantages of this method is to reduce the bias and prevent over fitting
